In [1]:
%matplotlib inline

In [2]:
#from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
K = keras.backend

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

In [4]:
def crooked_sigmoid(y_true, y_pred):
    """
    Our customized loss function
    Example: (4-class classification)
        y_pred = [ -500.0, -37.92, 99.99]
        y_true = [0, 0, 0]  # meaning that the true class is the fourth one.
    """
    # I guess y_pred.shape equals (m, 9) for Fashion MNIST
    s = tf.math.reduce_sum(y_pred, axis=1)
    #print(f"s.shape = {s.shape}")
    last_proba = 1 - tf.math.sigmoid(s)
    #print(f"tf.math.sigmoid(s) = {tf.math.sigmoid(s)}")
    #print(f"tf.math.sigmoid(s).shape = {tf.math.sigmoid(s).shape}")
    #print(f"last_proba = 1 - tf.math.sigmoid(s) = {last_proba}")
    #print(f"last_proba.shape = {last_proba.shape}")
    #first_probas = tf.math.sigmoid(s) * tf.nn.softmax(y_pred)
    #print()
    #first_probas = tf.stack([tf.math.sigmoid(s)]*tf.cast(tf.shape(y_pred)[1], tf.float32) , axis=1) * tf.nn.softmax(y_pred)
    #sigmoid_s_good_shape = tf.stack([tf.math.sigmoid(s)]*tf.cast(tf.shape(y_pred)[1], tf.float32) , axis=1)
    #print(f"tf.shape(y_pred) = {tf.shape(y_pred)}")
    #print(f"tf.shape(y_pred)[1] = {tf.shape(y_pred)[1]}")
    #print(f"tf.math.sigmoid(s) = {tf.math.sigmoid(s)}")
    #print(f"[tf.math.sigmoid(s)]*tf.shape(y_pred)[1] = {[tf.math.sigmoid(s)]*tf.shape(y_pred)[1]}")
    #print(f"[tf.math.sigmoid(s)]*tf.shape(y_pred)[1].numpy() = {[tf.math.sigmoid(s)]*tf.shape(y_pred)[1].numpy()}")
    
    #sigmoid_s_good_shape = tf.stack([tf.math.sigmoid(s)]*tf.shape(y_pred)[1].numpy(), axis=1)
    #sigmoid_s_good_shape = tf.stack([tf.math.sigmoid(s)]*y_pred.numpy().shape[1], axis=1)
    #sigmoid_s_good_shape = tf.stack([tf.math.sigmoid(s)]*K.eval(y_pred).shape[1], axis=1)
    sigmoid_s_good_shape = tf.stack([tf.math.sigmoid(s)]*y_pred.shape[1], axis=1)
    
    #print(f"sigmoid_s_good_shape = {sigmoid_s_good_shape}")
    #print(f"sigmoid_s_good_shape.shape = {sigmoid_s_good_shape.shape}")
    #print(f"tf.cast(tf.shape(y_pred)[1], tf.float32) = {tf.cast(tf.shape(y_pred)[1], tf.float32)}")
    first_probas = sigmoid_s_good_shape * tf.nn.softmax(y_pred)
    #print(f"y_pred[1] = {y_pred[1]}")
    #print(f"tf.nn.softmax(y_pred)[1] = {tf.nn.softmax(y_pred)[1]}")
    #print(f"first_probas = {first_probas}")
    #print(f"first_probas[1] = {first_probas[1]}")
    #print(f"first_probas.shape = {first_probas.shape}")
    #yy_pred = tf.concat([first_probas, [last_proba]], axis=0)
    #print(f"last_proba.shape = {last_proba.shape}")
    yy_pred = tf.concat([first_probas, last_proba[...,tf.newaxis]], axis=1)
    #print(f"yy_pred.shape = {yy_pred.shape}")
    print(f"yy_pred =\n{yy_pred}")
    #print(f"yy_true =\n{yy_true}")
    #print(f"tf.math.reduce_sum((yy_pred[:3], axis=1) = {tf.math.reduce_sum(yy_pred[:3], axis=1)}")
    ## Wrong!
    #yy_true = tf.cond(tf.math.reduce_sum(y_true, axis=1) < 1,
    #                  true_fn=lambda: tf.concat([y_true, [1]], 0),
    #                  false_fn=lambda: tf.concat([y_true, [0]], 0),
    #)
    yy_true = tf.concat([y_true, (1- tf.math.reduce_sum(y_true, axis=1))[...,tf.newaxis]], axis=1)
    #print(f"yy_true[:4] =\n{yy_true[:4]}")
    #return keras.metrics.categorical_crossentropy(
    #return keras.losses.categorical_crossentropy(
    #    yy_true,
    #    yy_pred,
    #    from_logits=False,
    #)
    return tf.nn.softmax_cross_entropy_with_logits(
        labels=yy_true,
        logits=yy_pred,
    )

In [5]:
model_nn = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(9)
])

model_nn.compile(optimizer='adam',
              loss=crooked_sigmoid,
              metrics=['accuracy'])

In [6]:
model_nn.fit(train_images, tf.one_hot(train_labels, depth=10)[:,:-1], epochs=10)

Epoch 1/10
yy_pred =
Tensor("crooked_sigmoid/concat:0", shape=(32, 10), dtype=float32)
yy_pred =
Tensor("crooked_sigmoid/concat:0", shape=(32, 10), dtype=float32)
1875/1875 [==============================] - 5s 3ms/step - loss: 1.7106 - accuracy: 0.6804
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 1.6245 - accuracy: 0.7549
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6103 - accuracy: 0.7644
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6030 - accuracy: 0.7711
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5978 - accuracy: 0.7752
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.5937 - accuracy: 0.7795
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 1.5899 - accuracy: 0.7821
Epoch 8/10
1875/1875 [==============================] - 5s 2ms/step - loss: 1.5844 - accuracy: 0.7876
Epoch 9/10
1875/1875 [==========

Is it because I implemented **wrongly** the loss function? Maybe the last return value should not be `tf.nn.softmax_cross_entropy_with_logits`

### Debug

In [ ]:
np.random.randint(0,10,size=(5))

In [ ]:
arti_true = np.random.rand(6000,9)
r = np.random.randint(0,9,size=(6,))
arti_true[range(6), r]

In [ ]:
r

In [ ]:
arti_true[:6]

In [ ]:
n_instances = 6000
n_classes = 9
arti_true = np.zeros((n_instances, n_classes))
arti_true[range(n_instances), np.random.randint(0, n_classes, size=(n_instances,))]
arti_pred = np.random.rand(*(arti_true.shape))
arti_pred.shape

In [ ]:
crooked_sigmoid(arti_true, arti_pred)

In [ ]:
crooked_sigmoid(arti_true, arti_true)

In [ ]:
def crooked_sigmoid(y_true, y_pred):
    """
    Our customized loss function
    Example: (4-class classification)
        y_pred = [ -500.0, -37.92, 99.99]
        y_true = [0, 0, 0]  # meaning that the true class is the fourth one.
    """
    # I guess y_pred.shape equals (m, 9) for Fashion MNIST
    s = tf.math.reduce_sum(y_pred, axis=1)
    last_proba = 1 - tf.math.sigmoid(s)
    sigmoid_s_good_shape = tf.stack([tf.math.sigmoid(s)]*y_pred.shape[1], axis=1)
    first_probas = sigmoid_s_good_shape * tf.nn.softmax(y_pred)
    yy_pred = tf.concat([first_probas, last_proba[...,tf.newaxis]], axis=1)
    print(f"yy_pred =\n{yy_pred}")
    yy_true = tf.concat([y_true, (1- tf.math.reduce_sum(y_true, axis=1))[...,tf.newaxis]], axis=1)
    print(f"yy_true =\n{yy_true}")
    #return tf.nn.softmax_cross_entropy_with_logits(
    #    labels=yy_true,
    #    logits=yy_pred,
    #)
    #return keras.metrics.categorical_crossentropy(
    return keras.losses.categorical_crossentropy(
        yy_true,
        yy_pred,
        from_logits=False,
    )

In [ ]:
crooked_sigmoid(arti_true, arti_true)

### Still buggy?
The loss in the cell above **shouldn't they be all zeros** (because we have already let `y_pred = y_true`)?

**No, actually not. Cf. below**

In [ ]:
crooked_sigmoid(arti_true, -999*np.ones((n_instances, n_classes)))

### Let's investigate the layers' activation functions

For example, let's try remove the `relu` in the hidden layer

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    #keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128),
    ## Try adding another layer, making it deeper.
    #keras.layers.Dense(128),
    keras.layers.Dense(9)
])

model.compile(optimizer='adam',
              loss=crooked_sigmoid,
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, tf.one_hot(train_labels, depth=10)[:,:-1], epochs=10)

In [ ]:
8/9

In [ ]:
9/10

### Write a customized loss function